In [ ]:
#@title Obtain Code Files from github
!rm -r Abnormal-Event-Detection-in-Surveillance-Video
!git clone https://github.com/Asmaa-S/Abnormal-Event-Detection-in-Surveillance-Video.git
#!git clone https://github.com/dina-adel/Abnormal-Event-Detection-in-Surveillance-Video

rm: cannot remove 'Abnormal-Event-Detection-in-Surveillance-Video': No such file or directory
Cloning into 'Abnormal-Event-Detection-in-Surveillance-Video'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (344/344), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 344 (delta 227), reused 231 (delta 119), pack-reused 0
Receiving objects: 100% (344/344), 74.27 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (227/227), done.


In [ ]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
#@title install packages [Not Currently needed]
!pip install sqlalchemy
!pip install superintendent
!pip install ModAL

In [ ]:
#@title imports { form-width: "20%" }
import sys

sys.path.insert(1, '/content/Abnormal-Event-Detection-in-Surveillance-Video/Code')

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle
from show_vid import *
from IPython.display import HTML
from base64 import b64encode
from time import sleep
import pickle as pkl
import matplotlib.pyplot as plt
from keras import optimizers
import os
from testing import *
from show_vid import show_vid
import numpy as np
from keras.models import load_model
from evaluate import *
import glob 
import cv2
from keras.layers import Dense, Activation, Lambda, Input, Concatenate,Flatten
import keras.backend as K 
from tensorflow.keras.utils import to_categorical


In [ ]:
#@title load trained model { form-width: "20%" }
job_uuid = '8b73974b-bd4e-4047-8fe6-fe585c69a0d1'      # ped1
#job_uuid = '2c14925e-d05a-4d0a-ab74-cb7a7bbd3db5'      # ped2
epoch = 50
val_loss =  -0.117306 # ped1
#val_loss = -0.049951 # ped2
time_length = 10
video_root_path= '/content/drive/MyDrive/Grad Project/data/UCSD'
dataset = 'UCSDped1'
#fetching paths to test_data, job_folder and trained model
test_dir = os.path.join(video_root_path, '{0}/testing_numpy'.format(dataset))
job_folder = os.path.join(video_root_path,dataset,'logs/jobs',job_uuid)
model_filename = 'model_snapshot_e{:03d}_{:.6f}.h5'.format(epoch, val_loss)

#load model
model = load_model(os.path.join(job_folder, model_filename))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 227, 227, 1)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 55, 55, 128)   15616     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 26, 26, 64)    204864    
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 10, 26, 26, 64)    295168    
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 10, 26, 26, 32)    110720    
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 10, 26, 26, 64)    221440    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 55, 55, 128)   204928

In [ ]:
#@title Modify architecture: Add Active learning layer- Wrap up as classifier { form-width: "20%" }
from keras.models import Model
import tensorflow as tf
def L2_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    out = tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))
    E =  tf.math.reduce_sum(out, axis=2, keepdims=False)
    sa = (E -K.min(E))/K.max(E)
    return sa #K.ones_like(sa)-sa

layers = [l for l in model.layers]
for l in layers:
    l.trainable = False
    #pass

input_layer = Input(shape = layers[0].output_shape[0][1:])
x = layers[1](input_layer)
x = layers[2](x)
x = layers[3](x)
latent = layers[4](x)
latent.trainable=True
x = layers[5](latent)
x = layers[6](x)
output = layers[7](x)
sa_layer = Lambda(L2_distance)([input_layer, output])

flat_sa = Flatten()(sa_layer)
flat_latent = Flatten()(latent)

concat_layer= Concatenate()([flat_latent, flat_sa])
UAI = Dense(1, activation ='sigmoid')(concat_layer)

UAI_model = Model(input_layer, UAI)



In [ ]:
#@title Compile { form-width: "20%" }
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

opt = optimizers.Adam(learning_rate=1e-6, decay=1e-4/100, epsilon=1e-6)
UAI_model.compile(loss= 'binary_crossentropy', optimizer=opt, metrics=['acc',precision_m, recall_m])


In [ ]:
UAI_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 227, 227 0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 10, 55, 55, 1 15616       input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 10, 26, 26, 6 204864      time_distributed[1][0]           
__________________________________________________________________________________________________
conv_lst_m2d (ConvLSTM2D)       (None, 10, 26, 26, 6 295168      time_distributed_1[1][0]         
______________________________________________________________________________________________

## Construct Datasets

In [ ]:
""" read numpy files from drive- extract ground truth labels- 
return train and test sets of Volumes and GT labels """

def chunck(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

def video_to_volumes(lst, n):
    """Yield successive n-sized chunks from lst of frames."""
    lst = np.expand_dims(lst, axis=-1)
    volumes = np.array(list(chunck(lst, n)))
   # print(volumes.shape)
    #volumes = np.swapaxes(volumes,3,1)
    
    return volumes
    
def construct_test_set(time_length=10):
    X = []
    y =[]
    np_directory = os.path.join(video_root_path, dataset,'testing_numpy')

    for i, video_npy in enumerate(os.listdir(np_directory)):
        #print (video_npy)
        test_data = np.load(os.path.join(np_directory,video_npy))
        if len(test_data)%time_length !=0:
            test_data= np.append(test_data, test_data[-(time_length-len(test_data)%time_length):], axis=0)


        volumes = video_to_volumes(test_data, time_length)
        abnormal_range = get_gt_range(video_root_path,dataset, i)
        frame_labels = [0]*len(test_data)
        for j in abnormal_range:
            frame_labels = np.array([1 if x+1>= j[0] and x+1<= j[1] else y for x, y in enumerate(frame_labels)])

        volume_labels = np.array(list(chunck(frame_labels, time_length))).any(axis=1)
        #print(frame_labels.shape)
        X.extend(volumes)
        y.extend(volume_labels)
    
    return np.array(X), np.array(y)


def construct_training_set(time_length=10):
    X = []
    np_directory = os.path.join(video_root_path, dataset,'training_numpy')

    for i, video_npy in enumerate(os.listdir(np_directory)):
       # print (video_npy)
        train_data = np.load(os.path.join(np_directory,video_npy))
        if len(train_data)%time_length !=0:
            train_data= np.append(train_data, train_data[-(time_length-len(train_data)%time_length):], axis=0)


        volumes = video_to_volumes(train_data, time_length)
        
        X.extend(volumes)
    
    return np.array(X)




In [ ]:
X, y = construct_test_set()
len(X), len(y)

(720, 720)

In [ ]:
X_train = construct_training_set()
len(X_train)

680

In [ ]:
X, y = shuffle (X, y)

X_train, y_train = shuffle(X_train, np.zeros(X_train.shape[0]))

## Sanity Check

In [ ]:
X_pool, y_pool = X[:200], y[:200]
X_test, y_test = X[200:], y[200:]


In [ ]:

#sanity check
EarlyStop = tf.keras.callbacks.EarlyStopping(patience =1, monitor= 'val_acc', restore_best_weights=True)
UAI_model.fit(
    validation_split =0.2,
    x=X_train[:50],
    batch_size=10,
    y=y_train[:50],
    epochs=1,
    callbacks = [EarlyStop] )

4/4 [==============================] - 46s 830ms/step - loss: 0.7053 - acc: 0.3167 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.6944 - val_acc: 0.5000 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


## Baseline performance: no active learning

In [ ]:
scores = UAI_model.evaluate(X_test, y_test, verbose=1)
print(f'accuracy = {scores[1]}')

17/17 [==============================] - 5s 134ms/step - loss: 0.6959 - acc: 0.4538 - precision_m: 0.5902 - recall_m: 0.4065
accuracy = 0.45384615659713745


In [ ]:
# output threshold
thresh = 0.48
ys = np.round(UAI_model.predict(X)[:,0], 2)
y_pred = np.array([i>thresh for i in ys])
cm = confusion_matrix(y, y_pred)
{'tn': cm[0, 0], 'fp': cm[0, 1], 'fn': cm[1, 0], 'tp': cm[1, 1]}


{'fn': 88, 'fp': 233, 'tn': 52, 'tp': 347}

##  active learning session

In [ ]:
# video display utilities

mean_frame = np.load(video_root_path+'/'+dataset+'/mean_frame_224.npy')

def display_video_volume(vid, mean_frame):
    show_vid(vid.squeeze(), mean_frame)

    mp4 = open('vid.mp4','rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

In [ ]:
#path = '/content/Abnormal-Event-Detection-in-Surveillance-Video/Code/handlabled_data.pkl'

# in case we we want to label part of the data, store it and come back to it, or append new labels on it later

path = 'handlabled_data.pkl'

if os.path.isfile(path):
    handlabled_data = pkl.load(open(path, 'rb'))
else:
    handlabled_data ={'X':[], 'y':[]}

len(handlabled_data['X'])

0

In [ ]:
#select abnormal-ranked pool for user-validation
all_scores = np.round(UAI_model.predict(X_pool)[:,0], 2)
abnormal_samples = X_pool[all_scores >thresh]
abnormal_samples.shape

(133, 10, 227, 227, 1)

In [ ]:
pkl.dump(abnormal_samples, open ( 'data.pkl', 'wb'))

In [ ]:
%store abnormal_samples

Stored 'abnormal_samples' (ndarray)


In [ ]:
start = 0 #len(handlabled_data['X']) 
print(f'starting at sample index = {start}')
budget  = 10

# start  AL looping
for i, samp in  enumerate( abnormal_samples[start:start+budget]):    
    if os.path.isfile('vid.mp4'):
        os. remove("vid.mp4")
    if os.path.isfile('vid.avi'):
         os. remove("vid.avi")
    sample = np.expand_dims(samp, axis =0)
    #score = UAI_model.predict(sample)[0][0]
    
    display_video_volume(sample[0].squeeze(), mean_frame)

    sleep(0.1)
    print('sample number', i)
    print('label with a single number- either 0 for normal sequence or 1 for abnormal: ')
    label = input("")

    os. remove("vid.mp4")
    os. remove("vid.avi")

    handlabled_data['X'].append(samp)
    handlabled_data['y'].append(int(label))


pkl.dump(handlabled_data, open(path, 'wb') )


starting at sample index = 0


sample number 0
label with a single number- either 0 for normal sequence or 1 for abnormal: 
1


sample number 1
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


sample number 2
label with a single number- either 0 for normal sequence or 1 for abnormal: 
1


sample number 3
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


...

sample number 5
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


sample number 6
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


sample number 7
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


sample number 8
label with a single number- either 0 for normal sequence or 1 for abnormal: 
1


sample number 9
label with a single number- either 0 for normal sequence or 1 for abnormal: 
0


## Retraining and evaluation after Active session

In [ ]:
 handlabled_data['X'] = np.array(handlabled_data['X'])
(handlabled_data['X']).shape

(167, 10, 227, 227, 1)

In [ ]:
X_labeled, y_labeled = shuffle(np.array(handlabled_data['X']),  np.array(handlabled_data['y']), random_state=0)

X_labeled.shape, y_labeled.shape

((167, 10, 227, 227, 1), (167,))

In [ ]:
y_labeled = np.array([int(x)>0 for x in y_labeled])


In [ ]:
EarlyStop = tf.keras.callbacks.EarlyStopping(patience =2, monitor = 'val_acc', restore_best_weights=True)
# Retrain on labeled data
hist = UAI_model.fit(
    validation_split= 0.2,
    x=X_labeled,
    batch_size=10,
    y=y_labeled,
    callbacks = [EarlyStop],
    epochs= 10)

Epoch 1/10
14/14 [==============================] - 6s 127ms/step - loss: 0.6952 - acc: 0.4511 - precision_m: 0.4738 - recall_m: 0.3035 - val_loss: 0.6948 - val_acc: 0.4412 - val_precision_m: 0.5417 - val_recall_m: 0.2167
Epoch 2/10
14/14 [==============================] - 2s 113ms/step - loss: 0.6939 - acc: 0.4436 - precision_m: 0.3440 - recall_m: 0.2597 - val_loss: 0.6932 - val_acc: 0.4706 - val_precision_m: 0.5667 - val_recall_m: 0.3083
Epoch 3/10
14/14 [==============================] - 2s 116ms/step - loss: 0.6918 - acc: 0.5263 - precision_m: 0.5814 - recall_m: 0.5088 - val_loss: 0.6897 - val_acc: 0.5294 - val_precision_m: 0.5000 - val_recall_m: 0.4611
Epoch 4/10
14/14 [==============================] - 2s 117ms/step - loss: 0.6890 - acc: 0.6090 - precision_m: 0.6303 - recall_m: 0.7776 - val_loss: 0.6870 - val_acc: 0.5000 - val_precision_m: 0.4722 - val_recall_m: 0.5806
Epoch 5/10
14/14 [==============================] - 2s 116ms/step - loss: 0.6870 - acc: 0.5940 - precision_m: 0.

In [ ]:
# X_train[10:], np.zeros(X_train[10:].shape[0])
scores = UAI_model.evaluate(X_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


17/17 [==============================] - 2s 129ms/step - loss: 0.6848 - acc: 0.5904 - precision_m: 0.6150 - recall_m: 0.8867
cosine_similarity: 59.04%


In [ ]:
# output threshold
ys = np.round(UAI_model.predict(X_test)[:,0], 2)
y_pred = np.array([i>thresh for i in ys])
cm = confusion_matrix(y_test, y_pred)
{'tn': cm[0, 0], 'fp': cm[0, 1], 'fn': cm[1, 0], 'tp': cm[1, 1]}


{'fn': 6, 'fp': 196, 'tn': 3, 'tp': 315}

## sanity check-2 viewing the original and reconstructed sequences

In [ ]:
#Viewing a  reconstructed  volume
vol = np.expand_dims(X[0], axis =0)
recon = model.predict(vol)
recon[0].shape

(10, 227, 227, 1)

In [ ]:
#reconstructed vol
display_video_volume(abnormal_samples[0].squeeze(), mean_frame)

In [ ]:
display_video_volume(recon[0].squeeze(), mean_frame/8)